In [1]:
import requests
import pandas as pd

class ScriptData:
    def __init__(self, api_key):
        self.api_key = api_key
        self.intraday_data = {}
    
    def fetch_intraday_data(self, script):
        # Fetching intraday data for the given script
        url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={script}&interval=1min&apikey={self.api_key}"
        response = requests.get(url)
        data = response.json()
        self.intraday_data[script] = data
    
    def convert_intraday_data(self, script):
        data = self.intraday_data[script]["Time Series (1min)"]
        df = pd.DataFrame(data)
        df = df.transpose()
        df.index = pd.to_datetime(df.index)
        df = df.astype({"1. open": float, "2. high": float, "3. low": float, "4. close": float, "5. volume": int})
        df = df.rename(columns={"1. open": "open", "2. high": "high", "3. low": "low", "4. close": "close", "5. volume": "volume"})
        self.intraday_data[script] = df
    
    def __getitem__(self, script):
        return self.intraday_data[script]
    
    def __setitem__(self, script, data):
        self.intraday_data[script] = data
    
    def __contains__(self, script):
        return script in self.intraday_data


In [2]:
def indicator1(df, timeperiod):
    indicator = df["close"].rolling(window=timeperiod).mean()
    result = pd.DataFrame({"timestamp": df.index, "indicator": indicator})
    return result


In [3]:
class Strategy:
    def __init__(self, script):
        self.script_data = ScriptData()
        self.df = self.script_data.fetch_intraday_data(script)
        self.indicator = indicator1(self.df, 5)
        self.signals = self.generate_signals()

    def generate_signals(self):
        signals = []
        for i in range(1, len(self.df)):
            if self.indicator["indicator"][i] > self.df["close"][i] and self.indicator["indicator"][i - 1] <= self.df["close"][i - 1]:
                signals.append("BUY")
            elif self.indicator["indicator"][i] < self.df["close"][i] and self.indicator["indicator"][i - 1] >= self.df["close"][i - 1]:
                signals.append("SELL")
            else:
                signals.append("NO_SIGNAL")
        signals = pd.DataFrame({"timestamp": self.df.index[1:], "signal": signals})
        return signals


In [4]:
import plotly.express as px

def plot_candlestick_chart(df):
    fig = px.candlestick(df, x='timestamp', open='open', high='high', low='low', close='close', title='Candlestick Chart')
    fig.show()

def plot_indicator_chart(indicator_data):
    fig = px.line(indicator_data, x='timestamp', y='indicator', title='Indicator Chart')
    fig.show()
